# Lab 3: SAP Gateway Integration with RAG-based Troubleshooting

Welcome to Lab 3! In this lab, we'll enhance our memory-enabled SAP agent by integrating with real SAP BTP APIs and adding Bedrock Knowledge Base for intelligent troubleshooting.

## 🎯 Learning Objectives

By the end of this lab, you will:
- Integrate with SAP BTP OData APIs for real sales order operations
- Implement Bedrock Knowledge Base for RAG-based troubleshooting
- Create intelligent tools that combine SAP data with knowledge retrieval
- Handle real-world SAP authentication and error scenarios

## ⏱️ Estimated Time: 35 minutes

In [10]:
# Import required libraries
import sys
import os
import json
import uuid
import logging
from datetime import datetime
from typing import Dict, Any, List, Optional

from utils import (
    print_header, print_success, print_error, print_info, print_warning,
    check_aws_credentials, create_mock_order_data, create_resource_name,
    display_architecture_progress, workshop_progress, wait_for_resource
)

# Display lab header
print_header("Lab 3: SAP Gateway Integration with RAG-based Troubleshooting")
display_architecture_progress(3)

🚀 Lab 3: SAP Gateway Integration with RAG-based Troubleshooting
----------------------------------------
📋 Current Architecture - Lab 3
----------------------------------------

Lab 3: Agent + Gateway + Memory
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   SAP Agent     │───▶│ AgentCore       │───▶│ SAP Systems     │
│                 │    │ Gateway         │    │ Email/KB        │
└─────────────────┘    └─────────────────┘    └─────────────────┘
         │
         ▼
┌─────────────────┐
│ AgentCore       │
│ Memory          │
└─────────────────┘
        


In [17]:
# Real SAP Sales Order API Integration with your credentials
import urllib3
import json
from datetime import datetime

# Your SAP Configuration
SAP_CONFIG = {
    "base_url": "https://beydev.ramadokr.people.aws.dev:8443",
    "username": "gyan", 
    "password": "Pass2025$"
}

class SAPSalesOrderService:
    """Real SAP Sales Order API Integration using API_SALES_ORDER_SRV"""
    
    def __init__(self, config):
        self.base_url = config["base_url"]
        self.username = config["username"]
        self.password = config["password"]
        self.csrf_token = None
        self.cookies = None
        self.http = urllib3.PoolManager()
        
        # SAP OData API endpoint
        self.api_base = f"{self.base_url}/sap/opu/odata/sap/API_SALES_ORDER_SRV"
        
        # Mock data as fallback
        self.mock_orders = [
            {
                "SalesOrder": "0000000001",
                "SoldToParty": "1000001", 
                "CustomerName": "ACME Corporation",
                "TotalNetAmount": "15000.00",
                "TransactionCurrency": "USD",
                "DeliveryBlockReason": "01",
                "DeliveryBlockReasonText": "Credit limit exceeded",
                "SalesOrderDate": "2024-01-15T00:00:00"
            },
            {
                "SalesOrder": "0000000002",
                "SoldToParty": "1000002",
                "CustomerName": "TechCorp Ltd", 
                "TotalNetAmount": "8500.00",
                "TransactionCurrency": "EUR",
                "DeliveryBlockReason": "02",
                "DeliveryBlockReasonText": "Incomplete documentation",
                "SalesOrderDate": "2024-01-16T00:00:00"
            },
            {
                "SalesOrder": "0000000003",
                "SoldToParty": "1000003",
                "CustomerName": "Global Industries",
                "TotalNetAmount": "22000.00", 
                "TransactionCurrency": "USD",
                "DeliveryBlockReason": "03",
                "DeliveryBlockReasonText": "Quality hold",
                "SalesOrderDate": "2024-01-17T00:00:00"
            },
            {
                "SalesOrder": "0000000004",
                "SoldToParty": "1000004",
                "CustomerName": "Manufacturing Inc",
                "TotalNetAmount": "12500.00",
                "TransactionCurrency": "USD", 
                "DeliveryBlockReason": "04",
                "DeliveryBlockReasonText": "Pricing approval required",
                "SalesOrderDate": "2024-01-18T00:00:00"
            },
            {
                "SalesOrder": "0000000005",
                "SoldToParty": "1000005",
                "CustomerName": "Retail Solutions",
                "TotalNetAmount": "9800.00",
                "TransactionCurrency": "EUR",
                "DeliveryBlockReason": "05", 
                "DeliveryBlockReasonText": "Customer payment overdue",
                "SalesOrderDate": "2024-01-19T00:00:00"
            }
        ]
        self.use_real_api = False
    
    def get_headers(self, include_csrf=False):
        """Get HTTP headers for SAP API calls"""
        headers = urllib3.make_headers(basic_auth=f"{self.username}:{self.password}")
        headers['Content-Type'] = 'application/json'
        headers['Accept'] = 'application/json'
        headers['DataServiceVersion'] = '2.0'
        headers['MaxDataServiceVersion'] = '2.0'
        
        if include_csrf and self.csrf_token:
            headers['x-csrf-token'] = self.csrf_token
            if self.cookies:
                headers['Cookie'] = self.cookies
        
        return headers
    
    def get_csrf_token(self):
        """Get CSRF token for write operations"""
        try:
            headers = self.get_headers()
            headers['x-csrf-token'] = 'fetch'
            
            url = f"{self.api_base}/$metadata"
            response = self.http.request('GET', url, headers=headers)
            
            if response.status == 200:
                self.csrf_token = response.headers.get('x-csrf-token', '')
                cookies = response.headers.get_all('Set-Cookie')
                if cookies:
                    self.cookies = '; '.join([cookie.split(';')[0] for cookie in cookies])
                print(f"✅ CSRF token obtained: {self.csrf_token[:10]}..." if self.csrf_token else "❌ No CSRF token")
                return True
            else:
                print(f"❌ Failed to get CSRF token: HTTP {response.status}")
                return False
        except Exception as e:
            print(f"❌ Error getting CSRF token: {e}")
            return False
    
 
    def get_sales_orders_with_delivery_blocks(self, top=5):
        """Get top N sales orders with delivery blocks"""
        
        if self.use_real_api:
            return self._get_real_blocked_orders(top)
        else:
            return self._get_mock_blocked_orders(top)
    
    def _get_real_blocked_orders(self, top=5):
        """Get blocked orders from real SAP API"""
        try:
            # OData query for sales orders with delivery blocks
            filter_query = "DeliveryBlockReason ne ''"
            select_fields = "SalesOrder,SoldToParty,TotalNetAmount,TransactionCurrency,DeliveryBlockReason,SalesOrderDate"
            
            url = f"{self.api_base}/A_SalesOrder?$filter={filter_query}&$select={select_fields}&$top={top}"
            
            headers = self.get_headers()
            print(f"🔍 Querying: {url}")
            
            response = self.http.request('GET', url, headers=headers)
            
            if response.status == 200:
                data = json.loads(response.data.decode('utf-8'))
                orders = data.get('d', {}).get('results', [])
                print(f"✅ Retrieved {len(orders)} blocked orders from SAP")
                return orders
            else:
                print(f"❌ API call failed: HTTP {response.status}")
                return []
        except Exception as e:
            print(f"❌ Error retrieving orders: {e}")
            return []
    
    def _get_mock_blocked_orders(self, top=5):
        """Get blocked orders from mock data"""
        print(f"ℹ️ Using mock data - returning top {top} blocked orders")
        return self.mock_orders[:top]
    
    def remove_delivery_block(self, sales_order_id, agent_identifier="SAP-Agent"):
        """Remove delivery block from specific sales order"""
        
        if self.use_real_api:
            return self._remove_real_delivery_block(sales_order_id, agent_identifier)
        else:
            return self._remove_mock_delivery_block(sales_order_id, agent_identifier)
    
    def _remove_real_delivery_block(self, sales_order_id, agent_identifier):
        """Remove delivery block using real SAP API"""
        try:
            # Get CSRF token first
            if not self.get_csrf_token():
                return {"success": False, "error": "Failed to get CSRF token"}
            
            # Update sales order to remove delivery block
            url = f"{self.api_base}/A_SalesOrder('{sales_order_id}')"
            
            update_data = {
                "DeliveryBlockReason": ""  # Empty string removes the block
            }
            
            headers = self.get_headers(include_csrf=True)
            
            print(f"🔧 Removing delivery block from order: {sales_order_id}")
            
            response = self.http.request(
                'PATCH', 
                url, 
                body=json.dumps(update_data).encode('utf-8'),
                headers=headers
            )
            
            if response.status in [200, 204]:
                # Add note about the removal
                note_result = self._add_order_note(sales_order_id, f"Delivery block removed by {agent_identifier} on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                
                return {
                    "success": True,
                    "message": f"Delivery block successfully removed from sales order {sales_order_id}",
                    "sales_order": sales_order_id,
                    "note_added": note_result,
                    "timestamp": datetime.now().isoformat()
                }
            else:
                return {
                    "success": False,
                    "error": f"Failed to remove delivery block: HTTP {response.status}",
                    "response": response.data.decode('utf-8') if response.data else None
                }
        except Exception as e:
            return {"success": False, "error": f"Error removing delivery block: {str(e)}"}
    
    def _remove_mock_delivery_block(self, sales_order_id, agent_identifier):
        """Remove delivery block from mock data"""
        for order in self.mock_orders:
            if order["SalesOrder"] == sales_order_id:
                if order.get("DeliveryBlockReason"):
                    original_reason = order.get("DeliveryBlockReasonText", "Unknown")
                    order["DeliveryBlockReason"] = ""
                    order["DeliveryBlockReasonText"] = ""
                    
                    return {
                        "success": True,
                        "message": f"Delivery block successfully removed from sales order {sales_order_id}",
                        "sales_order": sales_order_id,
                        "original_block_reason": original_reason,
                        "note_added": True,
                        "timestamp": datetime.now().isoformat()
                    }
                else:
                    return {"success": False, "error": f"Sales order {sales_order_id} has no active delivery block"}
        
        return {"success": False, "error": f"Sales order {sales_order_id} not found"}
    
    def _add_order_note(self, sales_order_id, note_text):
        """Add note to sales order (real API)"""
        try:
            url = f"{self.api_base}/A_SalesOrderText"
            
            note_data = {
                "SalesOrder": sales_order_id,
                "Language": "EN",
                "LongTextID": "0001",
                "LongText": note_text
            }
            
            headers = self.get_headers(include_csrf=True)
            
            response = self.http.request(
                'POST',
                url,
                body=json.dumps(note_data).encode('utf-8'),
                headers=headers
            )
            
            return response.status in [200, 201]
        except Exception as e:
            print(f"⚠️ Failed to add note: {e}")
            return False

# Initialize SAP Sales Order Service with your credentials
print("🚀 Initializing SAP Sales Order Service...")
print(f"📡 SAP URL: {SAP_CONFIG['base_url']}")
print(f"👤 Username: {SAP_CONFIG['username']}")

sap_service = SAPSalesOrderService(SAP_CONFIG)

# Test connection to your SAP system

print(f"📡 API Endpoint: {sap_service.api_base}")

🚀 Initializing SAP Sales Order Service...
📡 SAP URL: https://beydev.ramadokr.people.aws.dev:8443
👤 Username: gyan
📡 API Endpoint: https://beydev.ramadokr.people.aws.dev:8443/sap/opu/odata/sap/API_SALES_ORDER_SRV


In [18]:
# Import Bedrock Knowledge Base SDK
try:
    import boto3
    from botocore.exceptions import ClientError, NoCredentialsError
    
    # Initialize Bedrock Agent Runtime client for Knowledge Base queries
    bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')
    print_success("Bedrock Agent Runtime client initialized successfully")
    
except ImportError as e:
    print_error(f"Failed to import boto3: {e}")
    print_info("Creating mock Knowledge Base client for workshop demonstration...")
    
    # Create mock Knowledge Base client
    class MockBedrockAgentRuntime:
        def __init__(self):
            self.knowledge_base = {
                "credit limit": "To resolve credit limit issues: 1) Contact finance team to review customer credit status, 2) Request credit limit increase if justified, 3) Consider payment terms adjustment, 4) Escalate to sales manager for approval.",
                "incomplete documentation": "For incomplete documentation blocks: 1) Review missing documents list, 2) Contact customer for required paperwork, 3) Verify compliance requirements, 4) Update order once documents received.",
                "delivery block": "Delivery blocks prevent order fulfillment. Common causes: credit issues, incomplete docs, quality holds, or pricing problems. Always verify block reason before removal.",
                "sales order": "Sales orders represent customer purchase commitments. Key fields: order number, customer, items, delivery date, and status. Monitor for blocks and exceptions.",
                "troubleshooting": "SAP troubleshooting steps: 1) Identify the error/issue, 2) Check system logs, 3) Verify configuration, 4) Test with sample data, 5) Escalate if needed."
            }
        
        def retrieve_and_generate(self, **kwargs):
            query = kwargs.get('input', {}).get('text', '').lower()
            
            # Simple keyword matching for demo
            best_match = ""
            for key, value in self.knowledge_base.items():
                if key in query:
                    best_match = value
                    break
            
            if not best_match:
                best_match = "I don't have specific information about that topic. Please consult SAP documentation or contact support."
            
            return {
                'output': {
                    'text': best_match
                },
                'citations': [{
                    'generatedResponsePart': {
                        'textResponsePart': {
                            'text': best_match
                        }
                    },
                    'retrievedReferences': [{
                        'content': {
                            'text': best_match
                        },
                        'location': {
                            'type': 'S3',
                            's3Location': {
                                'uri': 'mock://knowledge-base/sap-troubleshooting.pdf'
                            }
                        }
                    }]
                }]
            }
    
    bedrock_agent_runtime = MockBedrockAgentRuntime()
    print_success("Mock Knowledge Base client created for workshop demonstration")

except NoCredentialsError:
    print_error("AWS credentials not found")
    print_info("Please configure AWS credentials or use mock services")

✅ Bedrock Agent Runtime client initialized successfully


In [19]:
# Configuration for SAP and Knowledge Base
SAP_CONFIG = {
    "base_url": "https://beydev.ramadokr.people.aws.dev:8443",
    "username": "gyan", 
    "password": "Pass2025$"
}

KNOWLEDGE_BASE_CONFIG = {
    "knowledge_base_id": "ABCDEFGHIJ",  # Replace with your Knowledge Base ID
    "model_arn": "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0"
}

print_info("Configuration loaded:")
print_info(f"SAP Base URL: {SAP_CONFIG['base_url']}")
print_info(f"Knowledge Base ID: {KNOWLEDGE_BASE_CONFIG['knowledge_base_id']}")

ℹ️  Configuration loaded:
ℹ️  SAP Base URL: https://beydev.ramadokr.people.aws.dev:8443
ℹ️  Knowledge Base ID: ABCDEFGHIJ


In [ ]:
# Initialize SAP Integration Service
print_header("Initializing SAP Integration Service", level=2)

try:
    SAP_CONFIG = {
    "base_url": "https://beydev.ramadokr.people.aws.dev:8443",
    "username": "gyan", 
    "password": "Pass2025$"
     }
    # Create SAP configuration
    sap_config{} = SAP_CONFIG()
    sap_config.sap_base_url = SAP_CONFIG["base_url"]
    sap_config.sap_username = SAP_CONFIG["username"]
    sap_config.sap_password = SAP_CONFIG["password"]
    sap_config.sap_timeout = SAP_CONFIG["timeout"]
    
    # Initialize SAP service
    sap_service = SAPIntegrationService(sap_config)
    
    # Test authentication
    print_info("Testing SAP authentication...")
    auth_success = sap_service.authenticate()
    
    if auth_success:
        print_success("SAP authentication successful!")
    else:
        print_warning("SAP authentication failed, using mock data")
        
except Exception as e:
    print_warning(f"SAP service initialization failed: {e}")
    print_info("Using mock SAP service for demonstration")
    sap_config = SAPConfig()
    sap_service = SAPIntegrationService(sap_config)

print_success("SAP Integration Service initialized!")

----------------------------------------
📋 Initializing SAP Integration Service
----------------------------------------
ℹ️  Testing SAP connection...
❌ SAP connection test failed: 'SAPSalesOrderService' object has no attribute 'test_connection'
ℹ️  Please check:
ℹ️     1. SAP system is accessible
ℹ️     2. Credentials are correct
ℹ️     3. API_SALES_ORDER_SRV service is available
ℹ️     4. Network connectivity to SAP system
⚠️  Continuing with workshop - some features may not work
✅ SAP Integration Service ready for use!


In [ ]:
# Create Knowledge Base RAG Service
class KnowledgeBaseRAGService:
    """Service for querying Bedrock Knowledge Base for SAP troubleshooting information."""
    
    def __init__(self, bedrock_client, knowledge_base_id: str, model_arn: str):
        self.bedrock_client = bedrock_client
        self.knowledge_base_id = knowledge_base_id
        self.model_arn = model_arn
    
    def query_knowledge_base(self, query: str, max_results: int = 5) -> Dict[str, Any]:
        """Query the knowledge base for relevant information."""
        try:
            response = self.bedrock_client.retrieve_and_generate(
                input={
                    'text': query
                },
                retrieveAndGenerateConfiguration={
                    'type': 'KNOWLEDGE_BASE',
                    'knowledgeBaseConfiguration': {
                        'knowledgeBaseId': self.knowledge_base_id,
                        'modelArn': self.model_arn,
                        'retrievalConfiguration': {
                            'vectorSearchConfiguration': {
                                'numberOfResults': max_results
                            }
                        }
                    }
                }
            )
            
            return {
                'answer': response['output']['text'],
                'sources': self._extract_sources(response.get('citations', []))
            }
            
        except Exception as e:
            print_error(f"Knowledge base query failed: {e}")
            return {
                'answer': "I'm unable to retrieve information from the knowledge base at the moment. Please consult SAP documentation or contact support.",
                'sources': []
            }
    
    def _extract_sources(self, citations: List[Dict]) -> List[str]:
        """Extract source references from citations."""
        sources = []
        for citation in citations:
            for ref in citation.get('retrievedReferences', []):
                location = ref.get('location', {})
                if location.get('type') == 'S3':
                    s3_uri = location.get('s3Location', {}).get('uri', '')
                    if s3_uri:
                        sources.append(s3_uri)
        return sources
    
    def get_troubleshooting_guidance(self, issue_type: str, context: str = "") -> str:
        """Get specific troubleshooting guidance for SAP issues."""
        query = f"How to troubleshoot {issue_type} in SAP sales orders? {context}"
        result = self.query_knowledge_base(query)
        
        answer = result['answer']
        sources = result['sources']
        
        if sources:
            answer += "\n\n📚 **Sources:**\n"
            for i, source in enumerate(sources[:3], 1):
                answer += f"{i}. {source.split('/')[-1]}\n"
        
        return answer

# Initialize Knowledge Base service
kb_service = KnowledgeBaseRAGService(
    bedrock_client=bedrock_agent_runtime,
    knowledge_base_id=KNOWLEDGE_BASE_CONFIG["knowledge_base_id"],
    model_arn=KNOWLEDGE_BASE_CONFIG["model_arn"]
)

print_success("Knowledge Base RAG Service initialized!")

In [ ]:
# Import memory components from Lab 2
try:
    from bedrock_agentcore.memory import MemoryClient
    print_success("AgentCore Memory SDK imported successfully")
except ImportError as e:
    print_error(f"Failed to import AgentCore Memory SDK: {e}")
    print_info("Using mock memory client from Lab 2...")
    
    # Use the same mock from Lab 2
    class MemoryClient:
        def __init__(self, region_name=None):
            self.conversations = {}
            self.region_name = region_name
        
        def create_memory_and_wait(self, **kwargs):
            import random
            import string
            name = kwargs.get('name', 'mock-memory')
            random_suffix = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
            return {
                'id': f'{name}-{random_suffix}',
                'name': name,
                'status': 'ACTIVE'
            }
        
        def get_last_k_turns(self, memory_id, actor_id, session_id, k=5):
            key = f'{actor_id}_{session_id}'
            return self.conversations.get(key, [])[-k:]
        
        def create_event(self, memory_id, actor_id, session_id, messages):
            key = f'{actor_id}_{session_id}'
            if key not in self.conversations:
                self.conversations[key] = []
            
            turn = []
            for message_text, role in messages:
                turn.append({
                    'role': role,
                    'content': {'text': message_text}
                })
            self.conversations[key].append(turn)
            return {'eventId': f'event-{len(self.conversations[key])}'}

# Initialize memory client
memory_client = MemoryClient(region_name="us-east-1")
memory = memory_client.create_memory_and_wait(name="SAPGatewayAgent")
memory_id = memory['id']

print_success(f"Memory client initialized with ID: {memory_id}")

In [ ]:
# Create Enhanced SAP Agent with Gateway Integration and RAG
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from strands.models import BedrockModel

# Configuration
ACTOR_ID = "sap_user_123"
SESSION_ID = "sap_gateway_session_001"

# Memory Hook Provider (from Lab 2)
class SAPMemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        try:
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                actor_id = ACTOR_ID
                session_id = SESSION_ID
            
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )
            
            if recent_turns:
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                print_info(f"✅ Loaded {len(recent_turns)} conversation turns")
                
        except Exception as e:
            print_error(f"Memory load error: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        messages = event.agent.messages
        try:
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                actor_id = ACTOR_ID
                session_id = SESSION_ID

            if messages and len(messages) > 0 and messages[-1]["content"][0].get("text"):
                message_text = messages[-1]["content"][0]["text"]
                message_role = messages[-1]["role"]
                
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(message_text, message_role)]
                )
                
                print_info(f"✅ Stored message with role: {message_role}")
                
        except Exception as e:
            print_error(f"Memory save error: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        print_info("✅ Memory hooks registered")

print_success("Enhanced SAP Memory Hook Provider created!")

In [ ]:
# Enhanced SAP Agent with Gateway Integration and RAG
SAP_GATEWAY_SYSTEM_PROMPT = f"""You are an advanced SAP Sales Order Agent with real SAP Gateway integration and intelligent troubleshooting capabilities.

You have access to:
1. **Real SAP BTP APIs** - Connect to live SAP systems for sales order operations
2. **Knowledge Base** - Access comprehensive SAP troubleshooting documentation
3. **Memory** - Remember previous conversations and context

Your capabilities include:
- List sales orders with delivery blocks from SAP system
- Get detailed order information with real-time data
- Remove delivery blocks with proper validation
- Provide intelligent troubleshooting guidance using RAG
- Send email notifications for critical actions

Always:
- Validate data with SAP before taking actions
- Provide troubleshooting guidance when issues arise
- Reference previous conversations when relevant
- Be professional and helpful

Today's date: {datetime.now().strftime('%Y-%m-%d')}
"""

class EnhancedSAPGatewayAgent:
    """Enhanced SAP Sales Order Agent with Gateway integration and RAG-based troubleshooting."""
    
    def __init__(self, sap_service, kb_service, memory_client, memory_id: str):
        self.sap_service = sap_service
        self.kb_service = kb_service
        self.memory_client = memory_client
        self.memory_id = memory_id
        
        # Create enhanced SAP tools with real integration
        @tool
        def list_blocked_orders_from_sap() -> str:
            """List all sales orders with delivery blocks from the SAP system."""
            try:
                blocked_orders = self.sap_service.get_sales_orders_with_delivery_blocks(
                    user_id=ACTOR_ID, 
                    session_id=SESSION_ID
                )
                
                if not blocked_orders:
                    return "✅ No sales orders with delivery blocks found in the SAP system."
                
                result = f"📋 Found {len(blocked_orders)} sales orders with delivery blocks:\n\n"
                
                for i, order in enumerate(blocked_orders, 1):
                    block_reason = order.delivery_block.block_reason if order.delivery_block else "Unknown"
                    result += f"{i}. **Order {order.order_id}**\n"
                    result += f"   - Customer: {order.customer_name}\n"
                    result += f"   - Value: {order.currency} {order.order_value:,.2f}\n"
                    result += f"   - Block Reason: {block_reason}\n\n"
                
                return result
                
            except Exception as e:
                error_msg = f"❌ Failed to retrieve blocked orders from SAP: {str(e)}"
                print_error(error_msg)
                return error_msg
        
        @tool
        def get_sap_order_details(order_id: str) -> str:
            """Get detailed information about a specific sales order from SAP system."""
            try:
                order = self.sap_service.get_sales_order_details(order_id)
                
                if not order:
                    return f"❌ Sales order {order_id} not found in SAP system."
                
                result = f"📦 **SAP Order Details for {order.order_id}**\n\n"
                result += f"🏢 **Customer Information:**\n"
                result += f"- Customer: {order.customer_name}\n"
                result += f"- Order Value: {order.currency} {order.order_value:,.2f}\n\n"
                
                if order.has_delivery_block and order.delivery_block:
                    result += f"🚫 **Active Delivery Block:**\n"
                    result += f"- Block Reason: {order.delivery_block.block_reason}\n"
                    result += f"- Status: Active\n\n"
                    
                    # Get troubleshooting guidance
                    guidance = self.kb_service.get_troubleshooting_guidance(
                        order.delivery_block.block_reason,
                        f"Sales order {order_id}"
                    )
                    result += f"💡 **Troubleshooting Guidance:**\n{guidance}\n"
                else:
                    result += f"✅ **No delivery blocks** - Order ready for processing\n"
                
                return result
                
            except Exception as e:
                error_msg = f"❌ Failed to retrieve order details from SAP: {str(e)}"
                print_error(error_msg)
                return error_msg
        
        @tool
        def remove_sap_delivery_block(order_id: str, reason: str = "Agent removal") -> str:
            """Remove delivery block from a sales order in the SAP system with validation."""
            try:
                # Remove block with comprehensive validation
                result = self.sap_service.remove_delivery_block_with_note(
                    order_id=order_id,
                    agent_identifier="Enhanced-SAP-Agent"
                )
                
                if result["success"]:
                    response = f"✅ **Delivery block successfully removed!**\n\n"
                    response += f"📋 **Order:** {order_id}\n"
                    response += f"🔓 **Original Block Reason:** {result['block_reason']}\n"
                    response += f"📝 **Note Added:** {'Yes' if result['note_added'] else 'No'}\n\n"
                    response += f"✨ The order is now released for delivery processing.\n"
                    
                    # Get post-removal guidance
                    guidance = self.kb_service.get_troubleshooting_guidance(
                        "delivery block removal",
                        f"Follow-up actions for order {order_id}"
                    )
                    response += f"\n📋 **Next Steps:**\n{guidance}"
                    
                    return response
                else:
                    return f"❌ Failed to remove delivery block: {result.get('message', 'Unknown error')}"
                
            except Exception as e:
                error_msg = f"❌ Failed to remove delivery block: {str(e)}"
                print_error(error_msg)
                
                # Get troubleshooting guidance for the error
                guidance = self.kb_service.get_troubleshooting_guidance(
                    "delivery block removal error",
                    f"Error: {str(e)}"
                )
                return f"{error_msg}\n\n💡 **Troubleshooting:**\n{guidance}"
        
        @tool
        def get_troubleshooting_help(issue_description: str) -> str:
            """Get intelligent troubleshooting guidance from the knowledge base."""
            try:
                guidance = self.kb_service.get_troubleshooting_guidance(
                    issue_description,
                    "SAP sales order troubleshooting"
                )
                
                return f"💡 **Troubleshooting Guidance:**\n\n{guidance}"
                
            except Exception as e:
                error_msg = f"❌ Failed to retrieve troubleshooting guidance: {str(e)}"
                print_error(error_msg)
                return error_msg
        
        @tool
        def send_notification_email(recipient: str, subject: str, message: str) -> str:
            """Send email notification for important SAP operations."""
            # Mock email functionality for workshop
            print_info(f"📧 Email notification sent to {recipient}")
            return f"✅ **Email sent successfully!**\n\n📧 **To:** {recipient}\n📋 **Subject:** {subject}\n📝 **Message:** {message[:100]}..."
        
        # Create Bedrock model
        bedrock_model = BedrockModel(
            model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
            temperature=0.3,
        )
        
        # Create enhanced agent with all capabilities
        self.agent = Agent(
            name="EnhancedSAPGatewayAgent",
            model=bedrock_model,
            system_prompt=SAP_GATEWAY_SYSTEM_PROMPT,
            hooks=[SAPMemoryHookProvider(memory_client, memory_id)],
            tools=[
                list_blocked_orders_from_sap,
                get_sap_order_details,
                remove_sap_delivery_block,
                get_troubleshooting_help,
                send_notification_email
            ],
            state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}
        )
    
    def process_message(self, message: str, session_id: str = None) -> str:
        """Process a user message with enhanced SAP and RAG capabilities."""
        try:
            if session_id:
                self.agent.state["session_id"] = session_id
            
            response = self.agent(message)
            return response.message
            
        except Exception as e:
            error_msg = f"Error processing message: {str(e)}"
            print_error(error_msg)
            return error_msg

print_success("Enhanced SAP Gateway Agent class created successfully!")

In [ ]:
# Initialize the Enhanced SAP Gateway Agent
print_header("Initializing Enhanced SAP Gateway Agent", level=2)

# Initialize the enhanced agent
gateway_agent = EnhancedSAPGatewayAgent(
    sap_service=sap_service,
    kb_service=kb_service,
    memory_client=memory_client,
    memory_id=memory_id
)

print_success("Enhanced SAP Gateway Agent initialized successfully!")
print_info(f"Memory ID: {memory_id}")
print_info(f"Actor ID: {ACTOR_ID}")
print_info(f"Session ID: {SESSION_ID}")
print_info("🚀 Ready for SAP Gateway operations with RAG-based troubleshooting!")

In [ ]:
# Test 1: List blocked orders from SAP system
print_header("Test 1: List Blocked Orders from SAP", level=2)

query1 = "Hello, I'm Sarah from the operations team. Can you show me all sales orders with delivery blocks from our SAP system?"
print(f"🧑‍💻 User: {query1}")
print("\n🤖 Enhanced SAP Agent:")
response1 = gateway_agent.process_message(query1)
print(response1)

In [ ]:
# Test 2: Get detailed order information with troubleshooting
print_header("Test 2: Order Details with Troubleshooting", level=2)

query2 = "Can you give me detailed information about order SO001234? I need to understand why it's blocked and how to resolve it."
print(f"🧑‍💻 User: {query2}")
print("\n🤖 Enhanced SAP Agent:")
response2 = gateway_agent.process_message(query2)
print(response2)

In [ ]:
# Test 3: Remove delivery block with validation
print_header("Test 3: Remove Delivery Block", level=2)

query3 = "The credit issue for SO001234 has been resolved. Please remove the delivery block and send a notification to operations@company.com."
print(f"🧑‍💻 User: {query3}")
print("\n🤖 Enhanced SAP Agent:")
response3 = gateway_agent.process_message(query3)
print(response3)

In [ ]:
# Test 4: RAG-based troubleshooting query
print_header("Test 4: RAG-based Troubleshooting", level=2)

query4 = "I'm having issues with incomplete documentation blocks. Can you provide troubleshooting guidance for this type of delivery block?"
print(f"🧑‍💻 User: {query4}")
print("\n🤖 Enhanced SAP Agent:")
response4 = gateway_agent.process_message(query4)
print(response4)

In [ ]:
# Test 5: Memory continuity with new session
print_header("Test 5: Memory Continuity", level=2)

new_session_id = "sap_gateway_session_002"
print_info(f"New Session ID: {new_session_id}")

query5 = "Hi, this is Sarah again. What orders did we work on earlier? I need to follow up on the actions we took."
print(f"🧑‍💻 User: {query5}")
print("\n🤖 Enhanced SAP Agent:")
response5 = gateway_agent.process_message(query5, new_session_id)
print(response5)

In [ ]:
# Test Knowledge Base directly
print_header("Knowledge Base Direct Test", level=2)

print_info("Testing Knowledge Base RAG service directly...")

# Test different types of queries
test_queries = [
    "credit limit exceeded",
    "incomplete documentation",
    "delivery block troubleshooting",
    "sales order processing"
]

for query in test_queries:
    print(f"\n🔍 **Query:** {query}")
    result = kb_service.query_knowledge_base(query)
    print(f"📝 **Answer:** {result['answer'][:200]}...")
    if result['sources']:
        print(f"📚 **Sources:** {len(result['sources'])} found")

print_success("Knowledge Base testing completed!")

## 🎉 Lab 3 Complete!

Congratulations! You've successfully created an enhanced SAP agent with:

### ✅ **Key Features Implemented:**

1. **🔗 SAP Gateway Integration**
   - Real SAP BTP API connections
   - OData client for sales order operations
   - Comprehensive error handling and validation

2. **🧠 RAG-based Troubleshooting**
   - Bedrock Knowledge Base integration
   - Intelligent troubleshooting guidance
   - Context-aware recommendations

3. **💾 Enhanced Memory**
   - Conversation continuity across sessions
   - Context-aware responses
   - Session management

4. **🛠️ Advanced Tools**
   - Real-time SAP data retrieval
   - Validated delivery block removal
   - Email notifications
   - Knowledge-based troubleshooting

### 🚀 **What's Next?**

Your enhanced SAP agent is now ready for production use with:
- Real SAP system integration
- Intelligent troubleshooting capabilities
- Memory-enabled conversations
- Comprehensive error handling

**Ready for Lab 4?** Continue building more advanced features like multi-agent workflows and advanced monitoring!

---

### 📋 **Configuration Checklist:**

For production deployment, ensure you have:
- [ ] Valid SAP BTP credentials and endpoint
- [ ] Bedrock Knowledge Base with SAP documentation
- [ ] AWS credentials configured
- [ ] AgentCore Memory service enabled
- [ ] Email service configured (SES/SNS)

🎊 **Excellent work completing Lab 3!**